## Final Project Submission

Please fill out:
* Student name: Brian Tracy
* Student pace: self paced
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


## EDA

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

raw_data = pd.read_csv('data\kc_house_data.csv')
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [15]:
np.mean(raw_data.price)

540296.5735055795

In [2]:
raw_data.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

We need to deal with NaN values.

- For 'yr_renovated', we assume that the house has never been renovated and set the value to 0.0 like other homes not renovated.

- For 'view', we assume that no value means there is no view. There are many 'NONE' values in this column, so we set the nulls to 'NONE'.

- For 'waterfront', we also assume that no value means no waterfront view, and set the value to 'NO'.

In [3]:
df = raw_data.copy()
df['yr_renovated'].fillna(0.0, inplace=True)
df['view'].fillna('NONE', inplace=True)
df['waterfront'].fillna('NO', inplace=True)

# dropping columns we will not need: id, lat, long, sqft_living15, sqft_lot15
# date, and zipcode
# we also will drop sqft_above (just use sqft_living)
# later decided to drop yr_renovated, will engineer new feature for renovation
df.drop(['id', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'zipcode',
         'date', 'sqft_above', 'yr_renovated'], axis=1, inplace=True)


## Baseline model

In [4]:
numerical_data = df.select_dtypes(include='number').copy()

In [5]:
numerical_data.corr()['price'].map(abs).sort_values(ascending=False)

price          1.000000
sqft_living    0.701917
bathrooms      0.525906
bedrooms       0.308787
floors         0.256804
sqft_lot       0.089876
yr_built       0.053953
Name: price, dtype: float64

Here decided to drop sqft_lot and yr_built for baseline model as well.

In [6]:
X = numerical_data.drop(['price', 'sqft_lot', 'yr_built'], axis=1)
y = numerical_data['price']

In [7]:
baseline_model = LinearRegression().fit(X,y).score(X,y)
baseline_model

0.5069513036458111

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                   random_state=0)

In [9]:
model_v1 = LinearRegression().fit(X_train, y_train)

In [10]:
model_v1.score(X_test,y_test)

0.4852484156831536

In [14]:
X.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'floors'], dtype='object')

In [11]:
model_v1.coef_

array([-57779.07216263,   5073.97505272,    316.02754652,  -2060.11712557])

In [12]:
model_v1.intercept_

70620.48702331533

In [16]:
from sklearn.model_selection import cross_validate, ShuffleSplit

splitter = ShuffleSplit(n_splits=3, test_size=0.25, random_state=0)

baseline_scores = cross_validate(
    estimator=model_v1,
    X=X_train,
    y=y_train,
    return_train_score=True,
    cv=splitter
)

print("Train score:", baseline_scores["train_score"].mean())
print("Test score:", baseline_scores["test_score"].mean())

Train score: 0.5141531604386964
Test score: 0.49984402999190375


Looks like a baseline model scores about 50, which we hope to improve by converting and adding back in categorical data first

## Convert categorical data

In [17]:
categorical_data = df.select_dtypes(exclude='number').copy()
categorical_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   waterfront     21597 non-null  object
 1   view           21597 non-null  object
 2   condition      21597 non-null  object
 3   grade          21597 non-null  object
 4   sqft_basement  21597 non-null  object
dtypes: object(5)
memory usage: 843.8+ KB


We have 6 categorical features to evaluate:
(technically 2 are numerical we are adapting)
- 'waterfront': engineer new boolean 'is_waterfront'
- 'grade': change to numeric
- 'sqft_basement': engineer new boolean 'has_basement'
- 'yr_renovated': engineer new boolean 'been_renovated'
    *note this uses the raw_data from original import

In [18]:
# set up new dataframe to concat later with numerical dataframe
converted_features = pd.DataFrame([])

# new 'is_waterfront' feature (boolean)
converted_features['is_waterfront'] = categorical_data.waterfront\
                                                     .map({'NO': 0, 'YES': 1})

# updated 'grade' feature
converted_features['grade'] = categorical_data.grade\
                                                  .map(lambda x: x.split()[0])

# 'sqft_basement' has some values of '?', before engineering new feature these
# must be converted to 0.0
categorical_data.sqft_basement.replace(to_replace='?', value=0.0, 
                                       inplace=True)
# then convert the whole column to float
categorical_data['sqft_basement'] = categorical_data.sqft_basement\
                                                    .astype('float')
# new 'has_basement' feature (boolean)
converted_features['has_basement'] = categorical_data.sqft_basement\
                                           .apply(lambda x: 1 if x > 1 else 0)

# new 'been_renovated' feature (boolean)
converted_features['been_renovated'] = raw_data.yr_renovated\
                                           .apply(lambda x: 1 if x > 0 else 0)

In [19]:
converted_features

,is_waterfront,grade,has_basement,been_renovated
0,0,7,0,0
1,0,7,1,1
2,0,6,0,0
3,0,7,1,0
4,0,8,0,0
...,...,...,...,...
21592,0,8,0,0
21593,0,8,0,0
21594,0,7,0,0
21595,0,8,0,0


- 'view': onehotencode (NONE, AVERAGE, GOOD, FAIR, EXCELLENT)
- 'condition': onehotencode (Average, Good, Very Good, Fair, Poor)

In [20]:
from sklearn.preprocessing import OneHotEncoder

categories = ['view', 'condition']
temp_df = categorical_data[categories].copy()

ohe = OneHotEncoder(sparse=False, drop='first')
ohe.fit(temp_df)

column_names = ohe.get_feature_names()

ohe_encoded = ohe.transform(temp_df)

temp_df_encoded = pd.DataFrame(ohe_encoded, columns=column_names)

In [22]:
temp_df.view.value_counts()

NONE         19485
AVERAGE        957
GOOD           508
FAIR           330
EXCELLENT      317
Name: view, dtype: int64

In [23]:
temp_df.condition.value_counts()

Average      14020
Good          5677
Very Good     1701
Fair           170
Poor            29
Name: condition, dtype: int64

In [21]:
temp_df_encoded

,x0_EXCELLENT,x0_FAIR,x0_GOOD,x0_NONE,x1_Fair,x1_Good,x1_Poor,x1_Very Good
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
21592,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
21593,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
21594,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
21595,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [24]:
merged_df = pd.concat([numerical_data, converted_features, temp_df_encoded],
                     axis=1)

In [25]:
merged_df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,yr_built,is_waterfront,grade,has_basement,been_renovated,x0_EXCELLENT,x0_FAIR,x0_GOOD,x0_NONE,x1_Fair,x1_Good,x1_Poor,x1_Very Good
0,221900.0,3,1.00,1180,5650,1.0,1955,0,7,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,538000.0,3,2.25,2570,7242,2.0,1951,0,7,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,180000.0,2,1.00,770,10000,1.0,1933,0,6,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,604000.0,4,3.00,1960,5000,1.0,1965,0,7,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,510000.0,3,2.00,1680,8080,1.0,1987,0,8,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,360000.0,3,2.50,1530,1131,3.0,2009,0,8,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
21593,400000.0,4,2.50,2310,5813,2.0,2014,0,8,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
21594,402101.0,2,0.75,1020,1350,2.0,2009,0,7,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
21595,400000.0,3,2.50,1600,2388,2.0,2004,0,8,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## Model Version 2

Model version 2 (bringing in converted categoricals)

In [26]:
X = merged_df.drop('price', axis=1)
y = merged_df['price']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                   random_state=0)

In [28]:
model_v2 = LinearRegression().fit(X_train,y_train)

In [29]:
model_v2.score(X_test,y_test)

0.6590650658767253

Looks like we are up to a score of 65 from 50

In [30]:
merged_df.corr()['price'].abs().sort_values(ascending=False)

price             1.000000
sqft_living       0.701917
bathrooms         0.525906
x0_NONE           0.356243
bedrooms          0.308787
x0_EXCELLENT      0.303059
is_waterfront     0.264306
floors            0.256804
x0_GOOD           0.182932
has_basement      0.178264
been_renovated    0.117543
x0_FAIR           0.092597
sqft_lot          0.089876
x1_Very Good      0.057531
yr_built          0.053953
x1_Fair           0.051432
x1_Good           0.030759
x1_Poor           0.019886
Name: price, dtype: float64

After looking at these results, decided to instead of onehotencode the view to engineer another boolean feature as to if the property has a view or not

In [39]:
trimmed_df = merged_df.iloc[: ,:-8]

In [40]:
trimmed_df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,yr_built,is_waterfront,grade,has_basement,been_renovated
0,221900.0,3,1.00,1180,5650,1.0,1955,0,7,0,0
1,538000.0,3,2.25,2570,7242,2.0,1951,0,7,1,1
2,180000.0,2,1.00,770,10000,1.0,1933,0,6,0,0
3,604000.0,4,3.00,1960,5000,1.0,1965,0,7,1,0
4,510000.0,3,2.00,1680,8080,1.0,1987,0,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...
21592,360000.0,3,2.50,1530,1131,3.0,2009,0,8,0,0
21593,400000.0,4,2.50,2310,5813,2.0,2014,0,8,0,0
21594,402101.0,2,0.75,1020,1350,2.0,2009,0,7,0,0
21595,400000.0,3,2.50,1600,2388,2.0,2004,0,8,0,0


In [42]:
# new 'has_view' feature (boolean)
trimmed_df['has_view'] = temp_df.view.apply(lambda x: 1 if x != 'NONE' else 0)

In [44]:
trimmed_df.has_view.value_counts()

0    19485
1     2112
Name: has_view, dtype: int64

We have a grade category already, going to try and not bring in the condition category for now and see what results we get.

In [45]:
X = trimmed_df.drop('price', axis=1)
y = trimmed_df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                   random_state=0)

model_v3 = LinearRegression().fit(X_train,y_train)

model_v3.score(X_test,y_test)

0.6569380471963555

Slightly worse than last model, but still hovering around a score of 65. Lets see if we do some feature ranking with RFE what we can generate

In [53]:
from sklearn.feature_selection import RFE

model_v4 = LinearRegression()

selector = RFE(model_v4, n_features_to_select=5)
selector = selector.fit(X_train,y_train.values.ravel())
selector.support_

array([False, False, False, False, False, False,  True,  True,  True,
        True,  True])

In [54]:
selected_columns = X_train.columns[selector.support_]
model_v4.fit(X_train[selected_columns], y_train)

LinearRegression()

In [55]:
model_v4.score(X_test[selected_columns],y_test)

0.552103921200882

Drops about 10 points to a score of 55 when we limit ourselves to 5 features